In [ ]:
import rasterio

In [ ]:
from rasterio.mask import mask
import geopandas as gpd
import fiona
import pandas as pd

In [ ]:
ras = 'data/stanford-td754wr4701-geotiff.tiff' # already interpolated but we will treat it as "data"

In [ ]:
shp = 'data/tl_2022_06073_faces.shp'

In [ ]:
gdf = gpd.read_file(shp)

In [ ]:
gdf.shape

In [ ]:
gdf.head()

In [ ]:
county = gdf.dissolve(by='COUNTYFP20')

In [ ]:
county.plot()

In [ ]:
rast = rasterio.open(ras)

In [ ]:
county = county.to_crs(rast.crs)

In [ ]:
county.plot()

In [ ]:
rast.crs

In [ ]:
coords = gdf.geometry
src = rast
df = county
import matplotlib.pyplot as plt
from rasterio.plot import show

In [ ]:
clipped_array, clipped_transform = mask(dataset=src, shapes=coords, crop=True)

df = df.to_crs(src.crs)
out_meta = src.meta.copy()
out_meta.update({"driver": "GTiff",
                 "height": clipped_array.shape[1],
                 "width": clipped_array.shape[2],
                 "transform": clipped_transform})
out_tif= "clipped_example.tif"
with rasterio.open(out_tif, "w", **out_meta) as dest:
    dest.write(clipped_array)
    
clipped = rasterio.open(out_tif)
fig, ax = plt.subplots(figsize=(8, 6))
p1 = df.plot(color=None,facecolor='none',edgecolor='red',linewidth = 2,ax=ax)
show(clipped, ax=ax)
ax.axis('off');

In [ ]:
clipped

In [ ]:
import rioxarray

In [ ]:
d = rioxarray.open_rasterio("clipped_example.tif")

In [ ]:
d

In [ ]:
d.plot()

In [ ]:
d.values.max()

In [ ]:
d.plot()

In [ ]:
type(d)

In [ ]:
d.dims

In [ ]:
d.values.mean()

In [ ]:
import numpy


In [ ]:
numpy.median(d.values)

In [ ]:
d.values.shape

In [ ]:
d.plot()

In [ ]:
d.plot.hist()

In [ ]:
type(d)



### Sampling the raster for "observations"

In [ ]:
import numpy
numpy.random.seed(12345)
sample_points = county.sample_points(50)

In [ ]:
m = county.explore()
sample_points.explore(m=m, color='red')

### Ensure sample points are separated by some threshold

In [ ]:
county.crs

In [ ]:
orig_crs = county.crs

In [ ]:
county_utm = county.to_crs(county.estimate_utm_crs())

In [ ]:
county_utm.plot()

In [ ]:
threshold = 10000 # no pair of stations within 10000 meters of each other

In [ ]:
n_points = 25 # number of stations desired

In [ ]:
thinning = True
sample_points = county_utm.sample_points(n_points * 4).explode(index_parts=True)
candidates = []
t2 = threshold**2
iter = 0
while thinning:
    p0 = numpy.random.choice(sample_points,1)[0]
    #p0 = sample_points[0]
    d0 = (sample_points.x - p0.x)**2 + (sample_points.y - p0.y)**2
    candidates.append(p0)
    if len(candidates) == n_points:
        thinning=False
    else:
        sample_points = sample_points[d0>t2]
    #print('iter: ', iter, 'shape sp: ', sampled_points.shape)
    iter += 1




In [ ]:
cp = gpd.GeoSeries(candidates)

In [ ]:
cp.plot()

In [ ]:
m = county_utm.plot()
cp.plot(ax=m, color='r');

In [ ]:
cp.crs = county_utm.crs
cp = cp.to_crs(county.crs)

In [ ]:
m = county.explore()
cp.explore(m=m, color='r')

In [ ]:
coord_list = [(x, y) for x, y in zip(cp.x, cp.y)]

observations = [x[0] for x in clipped.sample(coord_list)]
precip_gdf = gpd.GeoDataFrame(data=observations, columns=['inches'], geometry=cp)


In [ ]:
precip_gdf.plot(column='inches', legend=True);

In [ ]:
m = county.explore()
precip_gdf.explore(column='inches', m=m)

In [ ]:
precip_gdf.to_file("precip_sd.geojson", driver='GeoJSON')
county.to_file("sdcounty.geojson", driver='GeoJSON')

## Interpolation Methods

In [ ]:
tracts = gdf.dissolve(by='TRACTCE20')

tracts.shape

### Surface to Area Interpolation

#### Spatial Join on Centroid

In [ ]:
cents = tracts.centroid

In [ ]:
cents.plot()

In [ ]:
type(cents)

In [ ]:
coord_list = [(x, y) for x, y in zip(cents.x, cents.y)]
tracts['centest'] = [x[0] for x in clipped.sample(coord_list)]
tracts.head()

In [ ]:
tracts['centroid'] = tracts.centroid
tracts.set_geometry('centroid', inplace=True)

In [ ]:
tracts.plot(column='centest', legend=True);

In [ ]:
tracts.set_geometry('geometry', inplace=True)
tracts.plot(column='centest', legend=True);

#### Zonal Methods of Surface to Area Interpolation

In [ ]:
import rasterstats

In [ ]:
gdf.head()

In [ ]:
tracts.plot()

In [ ]:
from rasterstats import zonal_stats
tstats = zonal_stats(tracts, "clipped_example.tif",
            stats="count min mean max median")

#elevations2 = zonal_stats(
#    sd_tracts.to_crs(dem.rio.crs),  # Geotable with zones
#    "../data/nasadem/nasadem_sd.tif",  # Path to surface file
#)
#elevations2 = pandas.DataFrame(elevations2)

In [ ]:
tstats[:5]

In [ ]:
tstats = pd.DataFrame(tstats)

In [ ]:
tstats.head()

In [ ]:
tstats.shape

In [ ]:
tracts.shape

In [ ]:
tracts['mean'] = tstats['mean'].values
tracts.plot(column='mean', legend=True);

In [ ]:
tracts['median'] = tstats['median'].values
tracts.plot(column='median', legend=True);

In [ ]:
tracts['range'] = tstats['max'].values - tstats['min'].values
tracts.plot(column='range', legend=True);

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
sns.scatterplot(data=tracts, x='centest', y='mean')
plt.plot([10, 40], [10, 40]);

In [ ]:
sns.scatterplot(data=tracts, x='median', y='mean')
plt.plot([10, 40], [10, 40]);